In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import pandas as pd
import urllib.request, json

In [3]:
def API_Request(Stations, SubscriptionKey):

    #API Headers
    url = f"https://gateway.apiportal.ns.nl/Spoorkaart-API/api/v1/traject.geojson?stations={Stations}"
    headers = {
    'Cache-Control': 'no-cache',
    'Ocp-Apim-Subscription-Key': SubscriptionKey,
    }

    # Make the API request
    req = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(req)
    response_content = response.read().decode('utf-8')

    # Load the response content into JSON
    geojson_data = json.loads(response_content)

    # Convert the GeoJSON data to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

    return gdf


In [4]:
# function to make a dictionary with the seats form the transervices csv
def SeatSorter(TrainServicesData, TrainTravelData):
    # Inladen CSV met alle NS treindiensten
    TrainServices = pd.read_csv(TrainServicesData, delimiter = ';')

    # Dictionary maken met per treinserie een dataframe met ieder stukje tussen stations er in
    dataframes_dict = {}

    TrainServices.loc[0,'Code']
    TrainServices.loc[0,'String'].split(',')

    for i in range(len(TrainServices)):
        a = TrainServices.loc[i,'String'].split(',')
        df = pd.DataFrame({'From':[],'To':[],'Seats':[]})
        for j in range(len(a) - 1):
            new_row = pd.DataFrame({'From': [a[j]], 'To': [a[j + 1]], 'Seats':[None]})
            df = pd.concat([df, new_row], ignore_index=True)
        dataframes_dict[TrainServices.loc[i,'Code']] = df

        a = TrainServices.loc[i,'String'].split(',')
        a = a[::-1]
        df = pd.DataFrame({'From':[],'To':[],'Seats':[]})
        for j in range(len(a) - 1):
            new_row = pd.DataFrame({'From': [a[j]], 'To': [a[j + 1]], 'Seats':[None]})
            df = pd.concat([df, new_row], ignore_index=True)
        dataframes_dict[TrainServices.loc[i,'Code'] + 1] = df

    # Dictionary vullen met stoel aantallen
    for i in range(len(TrainTravelData)):
        if TrainTravelData.loc[i, 'JourneyNumber'] % 2 != 0:
            a = TrainTravelData.loc[i, 'JourneyNumber']//100
            c = 0
            for j in range(len(dataframes_dict[a*100+ 1])):
                if TrainTravelData.loc[i, 'UserStopCodeBegin'] == dataframes_dict[a*100+ 1].loc[j, 'From'].upper():
                    c = 1
                if c == 1:
                    if dataframes_dict[a*100 + 1].loc[j, 'Seats'] == None:
                        dataframes_dict[a*100+ 1].loc[j, 'Seats'] = TrainTravelData.loc[i, 'Seats']
                    else:
                        dataframes_dict[a*100+ 1].loc[j, 'Seats'] += TrainTravelData.loc[i, 'Seats']
                if TrainTravelData.loc[i, 'UserStopCodeEnd'] == dataframes_dict[a*100+ 1].loc[j, 'To'].upper():
                    break

        else:
            a = TrainTravelData.loc[i, 'JourneyNumber']//100
            c = 0
            for j in range(len(dataframes_dict[a*100])):
                if TrainTravelData.loc[i, 'UserStopCodeBegin'] == dataframes_dict[a*100].loc[j, 'From'].upper():
                    c = 1
                if c == 1:
                    if dataframes_dict[a*100].loc[j, 'Seats'] == None:
                        dataframes_dict[a*100].loc[j, 'Seats'] = TrainTravelData.loc[i, 'Seats']
                    else:
                        dataframes_dict[a*100].loc[j, 'Seats'] += TrainTravelData.loc[i, 'Seats']
                if TrainTravelData.loc[i, 'UserStopCodeEnd'] == dataframes_dict[a*100].loc[j, 'To'].upper():
                    break

    return dataframes_dict

In [5]:
# function to assign a color to a number based on where in the range of numbers it lies
def interpolate_color(lower_limit, upper_limit, lower_color, upper_color, number):
    # Clamp the number between the lower and upper limit
    number = max(min(number, upper_limit), lower_limit)
    
    # Calculate the interpolation factor
    factor = (number - lower_limit) / (upper_limit - lower_limit)
    
    # Interpolate each RGB component
    interpolated_color = tuple(
        int(lower_component + (upper_component - lower_component) * factor)
        for lower_component, upper_component in zip(lower_color, upper_color)
    )
    
    #normalize the color for usage
    color = [interpolated_color / 255.0 for interpolated_color in interpolated_color] 

    return color

In [6]:
#   NS Bron aanpassen
# Inladen CSV en toevoegen kolommen voor Stoelen en Bezette stoelen
TravelData = pd.read_csv('OC_NS_20241007.csv')
TravelData['Seats'] = 0
TravelData['Occupied Seats'] = 0

# Voor iedere treinserie het aantal stoelen toevoegen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'VehicleType'] == "VIRM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "DDZ":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "FLIRT FFF":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 53
    if TravelData.loc[i, 'VehicleType'] == "ICM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 75
    if TravelData.loc[i, 'VehicleType'] == "ICNG25":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 52
    if TravelData.loc[i, 'VehicleType'] == "SLT":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 54
    if TravelData.loc[i, 'VehicleType'] == "SNG":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 50
    if TravelData.loc[i, 'VehicleType'] == "SW7-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48
    if TravelData.loc[i, 'VehicleType'] == "SW9-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48

# Maar 1 week behouden dus 3 dagen weghalen uit de tabel
TravelData['OperatingDay'] = pd.to_datetime(TravelData['OperatingDay'])
dates_to_exclude = pd.to_datetime(['2024-10-14', '2024-10-15', '2024-10-16'])
df_filtered = TravelData[~TravelData['OperatingDay'].isin(dates_to_exclude)]
TravelData = df_filtered.reset_index(drop=True)

#treincodes boven 700000 aanpassen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'JourneyNumber'] > 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 700000

#treincode tussen 200000 en 700000 aanpassen
for i in range(len(TravelData)):
    if 200000 < TravelData.loc[i, 'JourneyNumber'] < 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 200000

In [7]:
a = SeatSorter('TrainServices.csv', TravelData)

In [8]:
df = pd.DataFrame({'From':[],'To':[],'Seats':[]})

for i in a.keys():
    b = a[i]
    d = 0
    c = 0
    for j in range(len(b)):
        if b.loc[j,'Seats'] == None:
            continue
        for k in range(len(df)):
            if b.loc[j,'From'] == df.loc[k, 'From'] and b.loc[j,'To'] == df.loc[k, 'To']:
                c = 1
                d = k
                break
            if b.loc[j,'To'] == df.loc[k, 'From'] and b.loc[j,'From'] == df.loc[k, 'To']:
                c = 1
                d = k
                break
        if c == 1:
            df.loc[d, 'Seats'] += b.loc[j,'Seats']
            c = 0
        else:
            new_row = pd.DataFrame({'From': [b.loc[j,'From']], 'To': [b.loc[j,'To']], 'Seats':[b.loc[j,'Seats']]})
            df = pd.concat([df, new_row], ignore_index=True)

In [12]:
display(df[:285])
print(df['Seats'][:285].max())
print(df['Seats'][:285].min())
df[:285].to_csv('test2.csv')
TravelData.to_csv('TravelData.csv')
#a.to_csv('SortedSeats.csv')
print(f'{df['From'][0]},{df['To'][0]}')

,From,To,Seats
0,Gn,Gerp,377446.0
1,Gerp,Hrn,377746.0
2,Hrn,Asn,375946.0
3,Asn,Bl,377421.0
4,Bl,Hgv,377421.0
...,...,...,...
280,Asdmm,Asd,54340.0
281,Wd,Bdg,265356.0
282,Bdg,Apn,265356.0
283,Apn,Ldl,268056.0


2933763.0
54340.0
Gn,Gerp


In [1]:
geometry_data = []

for i in range(len(df)):
    try: 
        geodata = API_Request(f'{df['From'][i]},{df['To'][i]}',SubscriptionKey)
        geometry_data.append(geodata['geometry'].iloc[0])
        print(i)
    except:
        SubscriptionKey = input('Subscriptionkey:')

NameError: name 'df' is not defined

In [ ]:
import urllib.request
import json
import geopandas as gpd

def API_Request(Stations, SubscriptionKey):
    # API Headers
    url = f"https://gateway.apiportal.ns.nl/Spoorkaart-API/api/v1/traject.geojson?stations={Stations}"
    headers = {
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': SubscriptionKey,
    }

    # Make the API request
    try:
        req = urllib.request.Request(url, headers=headers)
        response = urllib.request.urlopen(req)
        response_content = response.read().decode('utf-8')

        # Load the response content into JSON
        geojson_data = json.loads(response_content)

        # Convert the GeoJSON data to a GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

        return gdf
    except urllib.error.HTTPError as e:
        print(f"HTTPError: {e.code} - {e.reason}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Loop through the DataFrame and make API requests
geometry_data = []

for i in range(len(df)):
    while True:
        try:
            # Make the API request
            geodata = API_Request(f'{df["From"][i]},{df["To"][i]}', SubscriptionKey)
            
            # If the API request is successful, add geometry data and break out of the loop
            if geodata is not None:
                geometry_data.append(geodata['geometry'].iloc[0])
                print(i)
                break
            else:
                # If the API fails, prompt for a new subscription key
                SubscriptionKey = input('Invalid SubscriptionKey. Please enter a new SubscriptionKey: ')
        except Exception as e:
            print(f"An error occurred: {e}")
            SubscriptionKey = input('Error occurred. Please enter a new SubscriptionKey: ')
